In [1]:
import requests
import datetime
import time
import os
import csv
import pandas as pd
# from os import environ
from sqlalchemy import MetaData, create_engine, asc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

import psycopg2
from psycopg2 import Error

from flask import Flask, jsonify, request
from flask_cors import cross_origin, CORS
from models.models import Base

from models.models import ExpensesRaw, Filenames
# from endpointClasses.resources import Resources
#
from sqlalchemy import Column, Integer, BigInteger, String, Text, DateTime, \
    Float, Boolean, func, ForeignKeyConstraint, Index, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method
from datetime import datetime, timedelta, timezone
from geoalchemy2 import Geography, Geometry


In [2]:
run_all_flag = True

In [3]:
### Setup the application
app = Flask(__name__)

# Wrap CORS around the app so that the server does not block machine to machine
# or browser based requests
CORS(app)

In [4]:
# Engine below for Google Cloud PostgreSQL access.
engine = create_engine('postgresql+psycopg2://postgres:5413CrossFit2018@34'
                       '.70.40.80/transgov')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [5]:
col_names = ["Ministry", "Position", "Name", "Type", "Category", "Date", "Amount",
         "Description", "Receipt 1", "Receipt 2", "Receipt 3"]

In [6]:
# download  csv file
# Only do this once per day
print('Downloading .....')
df = pd.read_csv(r'https://expenses.alberta.ca/DownloadData.aspx?type=csv'
                   r'&d=IsVE/OcdpNZJ5rBbvji3qw', names=col_names,
                     low_memory=False, parse_dates=['Date'])

# Saving the dataframe
print('Saving to CSV ...')
filename = 'expenses/' + str( time.strftime('%Y%m%d') ) + ".csv"
df.to_csv(filename)
print('done...')

Saving to CSV ...
done...


In [7]:
print('Reading from file...')
# filename = "/" + filename
df=pd.read_csv(filename, names=col_names, low_memory=False, skiprows=2)
df_size = len(df.index)
print('Done...', df_size)

Reading from file...
Done... 536436


In [8]:
# compare the current download to the most previous download so that we can identify
# the changes and only process those.
# 1. Check file sizes
# 2. iterate through DF1 and lookup record in DF2
# 3.   if record exists then do nothing
# 4.   is this new record or a changed one? (hard to determine....we may have to manually see if dups become a problem)

# 4.   otherwise add record to database

print('Reading from previous file...')
# get the most recent entry in the table FileNames
prevFile = pd.read_sql(
        session.query(Filenames).statement, session.bind)

previousFile = prevFile._get_value(0,'name')

# open the file as a dataframe
prev_df = pd.read_csv(previousFile, names=col_names, low_memory=False, skiprows=2)
prev_df_size = len(prev_df.index)
print('Done...', prev_df_size)

Reading from previous file...
Done... 536436


In [9]:
# compare the two dataframes
# https://hackersandslackers.com/compare-rows-pandas-dataframes/
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    diff_df.to_csv('expenses/diff.csv')
    return diff_df

In [10]:
dataframe_difference(df, prev_df)

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [11]:
# save file name to filenames
filetosave = Filenames(
        name=filename,
        records = df_size,
        created_at=datetime.now()
    )
try:
    session.add(filetosave)
    session.commit()
except SQLAlchemyError as e:
    session.rollback()
    print('Could not save the new filename %s', filetosave)

In [12]:
shift_df = df[df.Date.str.startswith('$')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
56698,Agriculture and Forestry,"""Moore,Norine",Travel,Honorarium,12/13/2019,$7.35,Marketing Council Board Mtg - In Country Per Diem,NaN,NaN,NaN,NaN
56699,Agriculture and Forestry,"""Moore,Norine",Travel,Honorarium,12/13/2019,$11.60,Marketing Council Board Mtg - Lunch Allowance,NaN,NaN,NaN,NaN
56700,Agriculture and Forestry,"""Moore,Norine",Travel,Honorarium,12/13/2019,$20.75,Marketing Council Board Mtg - Dinner Allowance,NaN,NaN,NaN,NaN
56701,Agriculture and Forestry,"""Moore,Norine",Travel,Honorarium,12/13/2019,$471.67,Marketing Council Board Mtg - Mileage rtn from...,NaN,NaN,NaN,NaN
68923,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/21/2020,$27.27,Calgary Hearings - Mileage - 54KM,NaN,NaN,NaN,NaN
68924,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/21/2020,$12.75,Calgary Hearings - Parking - RGH,NaN,NaN,NaN,NaN
68925,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/28/2020,$30.30,Calgary Hearings - Mileage - 60KM,NaN,NaN,NaN,NaN
68927,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,03/06/2020,$47.47,Calgary Hearings - Mileage - 94KM,NaN,NaN,NaN,NaN
68928,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,03/06/2020,$12.75,Calgary Hearings - parking - SHC,NaN,NaN,NaN,NaN
69225,Health,"""Juhas,Michal",Travel,Honorarium,03/11/2020,$31.81,Edmonton Hearings - Mileage - 63KM,NaN,NaN,NaN,NaN


In [13]:
len(shift_df)

16

In [14]:
TOTAL_COLS = df.shape[1] -1
for idx, row in shift_df.iterrows():
    new_line = [df.iloc[idx, 0], None]
    for i in df.iloc[idx, 1: TOTAL_COLS]:
        new_line.append(i)
    df.loc[idx] = new_line


In [15]:
# Test
shift_df = df[df.Date.str.startswith('$')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [16]:
shift_df_2 = df[df.Date.str.startswith('H')]
shift_df_2

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
56701,Agriculture and Forestry,None,NaN,"Moore,Norine",Travel,Honorarium,12/13/2019,$114.44,Marketing Council Board Mtg - Accommodation 1 ...,http://expenses.alberta.ca/files/2020-02/exc_B...,NaN
68925,Health,None,NaN,"Sowa,Bernard, Dr.",Travel,Honorarium,02/28/2020,$14.25,Calgary Hearings - Parking - PLC,http://expenses.alberta.ca/files/2020-04/exc_4...,NaN
68928,Health,None,Vice-chair,"Wyrozub,Richard",Travel,Honorarium,02/19/2020,$50.50,Red Deer Hearings - Mileage - 100KM,NaN,NaN
69225,Health,None,Board Member,"Kreiner,Monica",Travel,Honorarium,02/24/2020,$253.51,Grande Prairie Hearings - Mileage - 502KM,NaN,NaN
69808,Health,None,Public Member,"Butler,Patricia",Travel,Honorarium,02/19/2020,$85.85,Red Deer Hearings - Mileage - 170KM,NaN,NaN
79588,Justice and Solicitor General,None,NaN,"Hak,Elizabeth",Travel,Honorarium,09/05/2019,$24.00,Rauf Hearing in Edmonton - Parking,http://expenses.alberta.ca/files/2020-02/exc_1...,NaN
179961,Executive Council,None,NaN,"McLeod,Patricia",Travel,Honorarium,06/17/2016,$117.60,AOE council strategy meeting - Medicine Hat - ...,http://expenses.alberta.ca/files/2016-10/exc_8...,NaN


In [17]:
len(shift_df_2)

7

In [18]:
TOTAL_COLS = df.shape[1] -1
shift_df_2
for idx, row in shift_df_2.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [19]:
# Test
shift_df = df[df.Date.str.startswith('H')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [20]:
shift_df_3 = df[df.Date.str.startswith('O')]
shift_df_3

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
79590,Agriculture and Forestry,None,Exec Director Wildfire Mgt. Br,"Tapp,North Darren James",Travel,Other Costs,11/20/2019,$7.35,Ottawa Trip -,NaN,NaN


In [21]:
len(shift_df_3)

1

In [22]:
TOTAL_COLS = df.shape[1] -1
shift_df_3
for idx, row in shift_df_3.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [23]:
# Test
shift_df = df[df.Date.str.startswith('O')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [24]:
shift_df_4 = df[df.Date.str.startswith('M')]
shift_df_4

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
179963,Education,None,Deputy Minister of Education,"Clarke,Curtis",Travel,Meals,10/11/2016,-$9.20,CMEC - Toronto - Breakfast (Adjustment),NaN,NaN


In [25]:
TOTAL_COLS = df.shape[1] -1
shift_df_4
for idx, row in shift_df_4.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [26]:
# Test
shift_df = df[df.Date.str.startswith('M')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


The next cells identify around 10000 records that were not saved in the initial run.  Not sure why, probably a DB connection issue

In [27]:
if run_all_flag:
    df.query('Name == "Filevich,Patricia"')

In [28]:
if run_all_flag:
    df.loc[509620:509630]

In [29]:
# new dataframe of missing records
if run_all_flag:
    missing_records_df = df.loc[498839: 509627]

In [30]:
if run_all_flag:
    missing_records_df.head()

In [31]:
if run_all_flag:
    missing_records_df.tail()

In [ ]:
if run_all_flag:
    len(missing_records_df)

Duplicates

In [32]:
# identify duplicate values
dups_df= df[df.duplicated(keep=False)] # identifies 850
# dups_df= df[df.duplicated()] # Identifies 431

In [33]:
dups_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
5166,Culture,Executive Director Arts Branch,"Anderson,Jeffrey",Travel,Mileage,10/10/2012,$1.52,Travel in Edmonton to departmental meeting,NaN,NaN,NaN
5168,Culture,Executive Director Arts Branch,"Anderson,Jeffrey",Travel,Mileage,10/10/2012,$1.52,Travel in Edmonton to departmental meeting,NaN,NaN,NaN
6193,Human Services,"Member , PDD South Board","Switzer,Lea",Travel,Honorarium,06/13/2013,$9.20,Board Governance / Edmonton - Breakfast Per Diem,NaN,NaN,NaN
6504,Energy,"Exec. Dir., Can Eng Strat Secr","Klashinsky,Sandra",Travel,Meals,09/24/2013,$9.20,Meetings -,NaN,NaN,NaN
7374,Justice and Solicitor General,Executive Director,"Merryweather,Lorne",Travel,Ground Transportation,01/31/2014,$3.20,ETS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
533983,Human Services,"Member, Lesser Slave Lake CAP","Jackson,Betty J",Travel,Honorarium,02/25/2013,$20.75,Expense to attend appeals - Dinner,NaN,NaN,NaN
533985,Human Services,"Member, Lesser Slave Lake CAP","Jackson,Betty J",Travel,Honorarium,02/25/2013,$11.60,Expense to attend appeals - Lunch,NaN,NaN,NaN
534937,Human Services,Assistant Deputy Minister,"Ludvigsen,Donna M",Travel,Mileage,10/29/2012,$15.66,Grande Prairie AB Works Visit,NaN,NaN,NaN
535602,Enterprise and Advanced Education,SFB Member,"Ruptash,Lillian",Travel,Honorarium,03/21/2013,$20.75,SFB Meeting in Edmonton - Supper,NaN,NaN,NaN


In [ ]:
# FUTURE only process delta dataframe!!!


# Process the dataframe
# Note that after the very first attempt at this we need to test for whether the record already exist and then update the
# date last found

start_time = time.time()
length = len(df)
ctr = 0

for index, row in df.iterrows():
    
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
    newAmount = df._get_value(index, 'Amount').replace('$', '').replace(',','')
    newAmount = float(newAmount)

    if len(str(df._get_value(index, 'Receipt 1'))) > 5:
        newReceipt1 = df._get_value(index, 'Receipt 1')
    else:
        newReceipt1 = ''
    if len(str(df._get_value(index, 'Receipt 2'))) > 5:
        newReceipt2 = df._get_value(index, 'Receipt 2')
    else:
        newReceipt2 = ''
    if len(str(df._get_value(index, 'Receipt 3'))) > 5:
        newReceipt3 = df._get_value(index, 'Receipt 3')
    else:
        newReceipt3 = ''  
    
    expense = ExpensesRaw(
        ministry=df._get_value(index, 'Ministry'),
        position=df._get_value(index, 'Position'),
        name=df._get_value(index, 'Name'),
        type=df._get_value(index, 'Type'),
        category=df._get_value(index, 'Category'),
        expense_date=newdate,
        amount=newAmount,
        description=df._get_value(index, 'Description'),
        receipt1=newReceipt1,
        receipt2=newReceipt2,
        receipt3=newReceipt3,
        date_last_found=datetime.now(),
        created_at=datetime.now(),
        updated_at=datetime.now(),
    )
    
#     qry = session.query(ExpensesRaw)
#     qry = qry.filter(
#                      ExpensesRaw.ministry==df._get_value(index, 'Ministry'),
#                      ExpensesRaw.position==df._get_value(index, 'Position'),
#                      ExpensesRaw.name==df._get_value(index, 'Name'),
#                      ExpensesRaw.type==df._get_value(index, 'Type'),
#                      ExpensesRaw.category==df._get_value(index, 'Category'),
#                      ExpensesRaw.expense_date==newdate,
#                      ExpensesRaw.amount==newAmount,
#                      ExpensesRaw.description==df._get_value(index, 'Description'),
#                      ExpensesRaw.receipt1==df._get_value(index, 'Receipt 1'),
#                      ExpensesRaw.receipt2==df._get_value(index, 'Receipt 2'),
#                      ExpensesRaw.receipt3==df._get_value(index, 'Receipt 3')
#                     ).first()
#     print('Query', qry)
    
    if ctr % 100 == 0:
        perc = "{:.3f}".format(ctr/length)
        elapsed = "{:.2f}".format(time.time() - start_time)
        print('Count: %s of %s percentage %s elapsed %s ' %(ctr, length, perc, elapsed))
    
    ctr = ctr + 1
    
    try:
        session.add(expense)
        session.commit()
    except SQLAlchemyError as e:
#         error = str(e.__dict__['orig'])
        session.rollback()
        print('Could not save the new expense %s',
              expense)
#         print(error)
   

In [36]:
# for each row in dups_df find it in the expensesraw table of the db and update the isduplicated flag to true
ctr = 1
for index, row in dups_df.iterrows():
    #find the row in the database
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
    newAmount = df._get_value(index, 'Amount').replace('$', '').replace(',','')
    newAmount = float(newAmount)

    if len(str(df._get_value(index, 'Receipt 1'))) > 5:
        newReceipt1 = df._get_value(index, 'Receipt 1')
    else:
        newReceipt1 = ''
    if len(str(df._get_value(index, 'Receipt 2'))) > 5:
        newReceipt2 = df._get_value(index, 'Receipt 2')
    else:
        newReceipt2 = ''
    if len(str(df._get_value(index, 'Receipt 3'))) > 5:
        newReceipt3 = df._get_value(index, 'Receipt 3')
    else:
        newReceipt3 = ''
        
    print('Position : ',  df._get_value(index, 'Position'))
    newPosition = df._get_value(index, 'Position')
    
    if df.isnull()._get_value(index, 'Position'):
        print('Position is None')
        newPosition = ''
    
    qry = session.query(ExpensesRaw)
    qry = qry.filter(
                     ExpensesRaw.ministry==df._get_value(index, 'Ministry'),
                     ExpensesRaw.position==newPosition,
                     ExpensesRaw.name==df._get_value(index, 'Name'),
                     ExpensesRaw.type==df._get_value(index, 'Type'),
                     ExpensesRaw.category==df._get_value(index, 'Category'),
                     ExpensesRaw.expense_date==newdate,
                     ExpensesRaw.amount==newAmount,
                     ExpensesRaw.description==df._get_value(index, 'Description'),
                     ExpensesRaw.receipt1==newReceipt1,
                     ExpensesRaw.receipt2==newReceipt2,
                     ExpensesRaw.receipt3==newReceipt3,
                    ).all()
    for result in qry:
        result.is_duplicated = True
#         print(result)
    try:
        session.commit()
    except:
        print('Could not update')
    print('Count', ctr)
    ctr = ctr + 1    
    

Position :  Executive Director Arts Branch
Count 1
Position :  Executive Director Arts Branch
Count 2
Position :  Member , PDD South Board
Count 3
Position :  Exec. Dir., Can Eng Strat Secr
Count 4
Position :  Executive Director
Count 5
Position :  PT Commissioners, WCB Appeals
Count 6
Position :  PT Commissioners, WCB Appeals
Count 7
Position :  Mbr, AISH & IES Appeal Panels
Count 8
Position :  Member
Count 9
Position :  Member
Count 10
Position :  Mbr, AISH & IESA Appeal Panels
Count 11
Position :  ADM, Treasury Management
Count 12
Position :  ED/Commissioner Cul Industries
Count 13
Position :  Senior Financial Officer
Count 14
Position :  Regional Ex Dir-South
Count 15
Position :  Dir-Access and Service Del
Count 16
Position :  Minister
Count 17
Position :  Minister
Count 18
Position :  Executive Dir. Major Cap. Proj
Count 19
Position :  Exec. Dir. Policy & Info.
Count 20
Position :  Exec. Dir. Policy & Info.
Count 21
Position :  Exec. Dir. Policy & Info.
Count 22
Position :  Region

Count 194
Position :  Regional Dir, Upper/Low Peace
Count 195
Position :  Assistant Deputy Minister
Count 196
Position :  Board Member
Count 197
Position :  Board Member
Count 198
Position :  Vice-chair
Count 199
Position :  Vice-chair
Count 200
Position :  Member
Count 201
Position :  Member
Count 202
Position :  Vice Chair and Member
Count 203
Position :  Vice Chair and Member
Count 204
Position :  ED, Specialized Pros.
Count 205
Position :  ED, Specialized Pros.
Count 206
Position :  Regional Dir, Upper/Low Peace
Count 207
Position :  Regional Dir, Upper/Low Peace
Count 208
Position :  Chief Scientist and ADM
Count 209
Position :  Chief Scientist and ADM
Count 210
Position :  Regional Ex Dir-South
Count 211
Position :  Regional Ex Dir-South
Count 212
Position :  Board Member
Count 213
Position :  Board Member
Count 214
Position :  Member
Count 215
Position :  Member
Count 216
Position :  Executive Director, Mon & Obs.
Count 217
Position :  Executive Director, Mon & Obs.
Count 218
Po

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: character varying = double precision
LINE 3: ...esraw.amount =  -5.25 AND expensesraw.description = 'NaN'::f...
                                                             ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: SELECT expensesraw.id AS expensesraw_id, expensesraw.ministry AS expensesraw_ministry, expensesraw.position AS expensesraw_position, expensesraw.name AS expensesraw_name, expensesraw.type AS expensesraw_type, expensesraw.category AS expensesraw_category, expensesraw.expense_date AS expensesraw_expense_date, expensesraw.amount AS expensesraw_amount, expensesraw.description AS expensesraw_description, expensesraw.receipt1 AS expensesraw_receipt1, expensesraw.receipt2 AS expensesraw_receipt2, expensesraw.receipt3 AS expensesraw_receipt3, expensesraw.is_duplicated AS expensesraw_is_duplicated, expensesraw.date_last_found AS expens

In [ ]:
print(df.head())